In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/star/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 500
test_batch = 500
lr = 0.04
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0/to_pggan/2000shot/cm' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.4, 'drop_connect_rate':0.4})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/star/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)    
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            lam = np.random.beta(cm_beta, cm_beta)
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1]*inputs.size()[-2]))
            
            outputs = student_model(inputs)
            loss_main = (criterion(outputs, target_a) * lam) + (criterion(outputs, target_b) * (1. - lam))
            
        else:
            outputs = student_model(inputs)
            loss_main = criterion(outputs, targets)

        # compute output
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
loss_list = []
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

    loss_list.append([train_loss, test_loss, source_loss])
    
    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    if (epoch+1) % 50 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.040000
Train | 8/8 | Loss:1.3597 | MainLoss:1.1163 | Alpha:0.1131 | SPLoss:0.2850 | CLSLoss:1.8654 | top1:56.2750 | AUROC:0.0000
Test | 65/8 | Loss:0.5964 | MainLoss:0.5964 | SPLoss:0.3534 | CLSLoss:1.7399 | top1:74.9813 | AUROC:0.8383
Test | 62/8 | Loss:0.4479 | MainLoss:0.4479 | SPLoss:0.3534 | CLSLoss:1.7399 | top1:81.3696 | AUROC:0.9905

Epoch: [2 | 1000] LR: 0.052000
Train | 8/8 | Loss:0.8516 | MainLoss:0.6186 | Alpha:0.1163 | SPLoss:0.3364 | CLSLoss:1.6639 | top1:69.0250 | AUROC:0.0000
Test | 65/8 | Loss:0.4874 | MainLoss:0.4874 | SPLoss:0.3193 | CLSLoss:1.5820 | top1:84.8442 | AUROC:0.9327
Test | 62/8 | Loss:0.4210 | MainLoss:0.4210 | SPLoss:0.3193 | CLSLoss:1.5820 | top1:80.0000 | AUROC:0.9900

Epoch: [3 | 1000] LR: 0.064000
Train | 8/8 | Loss:0.7697 | MainLoss:0.5584 | Alpha:0.1162 | SPLoss:0.3122 | CLSLoss:1.5049 | top1:72.6500 | AUROC:0.0000
Test | 65/8 | Loss:0.3382 | MainLoss:0.3382 | SPLoss:0.3137 | CLSLoss:1.4156 | top1:91.3115 | AUROC:0.9767
Tes

Test | 65/8 | Loss:0.2567 | MainLoss:0.2567 | SPLoss:0.3550 | CLSLoss:0.1051 | top1:93.4704 | AUROC:0.9979
Test | 62/8 | Loss:0.3346 | MainLoss:0.3346 | SPLoss:0.3550 | CLSLoss:0.1051 | top1:86.3270 | AUROC:0.9949

Epoch: [24 | 1000] LR: 0.159943
Train | 8/8 | Loss:0.5331 | MainLoss:0.4824 | Alpha:0.1100 | SPLoss:0.3545 | CLSLoss:0.1058 | top1:77.9500 | AUROC:0.0000
Test | 65/8 | Loss:0.1642 | MainLoss:0.1642 | SPLoss:0.3798 | CLSLoss:0.1233 | top1:97.7726 | AUROC:0.9976
Test | 62/8 | Loss:0.4622 | MainLoss:0.4622 | SPLoss:0.3798 | CLSLoss:0.1233 | top1:73.9613 | AUROC:0.9937

Epoch: [25 | 1000] LR: 0.159933
Train | 8/8 | Loss:0.5644 | MainLoss:0.5121 | Alpha:0.1144 | SPLoss:0.3512 | CLSLoss:0.1056 | top1:74.0000 | AUROC:0.0000
Test | 65/8 | Loss:0.2038 | MainLoss:0.2038 | SPLoss:0.3553 | CLSLoss:0.1226 | top1:95.1059 | AUROC:0.9969
Test | 62/8 | Loss:0.3337 | MainLoss:0.3337 | SPLoss:0.3553 | CLSLoss:0.1226 | top1:84.6101 | AUROC:0.9949

Epoch: [26 | 1000] LR: 0.159923
Train | 8/8 | L

Test | 62/8 | Loss:0.5071 | MainLoss:0.5071 | SPLoss:0.3921 | CLSLoss:0.0915 | top1:69.0400 | AUROC:0.9942

Epoch: [46 | 1000] LR: 0.159544
Train | 8/8 | Loss:0.5665 | MainLoss:0.5111 | Alpha:0.1158 | SPLoss:0.3788 | CLSLoss:0.0998 | top1:74.5750 | AUROC:0.0000
Test | 65/8 | Loss:0.2776 | MainLoss:0.2776 | SPLoss:0.3489 | CLSLoss:0.0816 | top1:93.8505 | AUROC:0.9973
Test | 62/8 | Loss:0.3471 | MainLoss:0.3471 | SPLoss:0.3489 | CLSLoss:0.0816 | top1:88.2765 | AUROC:0.9957

Epoch: [47 | 1000] LR: 0.159517
Train | 8/8 | Loss:0.5177 | MainLoss:0.4661 | Alpha:0.1103 | SPLoss:0.3635 | CLSLoss:0.1043 | top1:77.8500 | AUROC:0.0000
Test | 65/8 | Loss:0.1666 | MainLoss:0.1666 | SPLoss:0.3947 | CLSLoss:0.1220 | top1:97.7882 | AUROC:0.9980
Test | 62/8 | Loss:0.4256 | MainLoss:0.4256 | SPLoss:0.3947 | CLSLoss:0.1220 | top1:76.4744 | AUROC:0.9950

Epoch: [48 | 1000] LR: 0.159489
Train | 8/8 | Loss:0.5420 | MainLoss:0.4855 | Alpha:0.1108 | SPLoss:0.3898 | CLSLoss:0.1197 | top1:77.2000 | AUROC:0.0000


Train | 8/8 | Loss:0.4976 | MainLoss:0.4665 | Alpha:0.4058 | SPLoss:0.0112 | CLSLoss:0.0655 | top1:73.5250 | AUROC:0.0000
Test | 65/8 | Loss:0.2205 | MainLoss:0.2205 | SPLoss:0.0117 | CLSLoss:0.0675 | top1:96.0997 | AUROC:0.9986
Test | 62/8 | Loss:0.3502 | MainLoss:0.3502 | SPLoss:0.0117 | CLSLoss:0.0675 | top1:86.3139 | AUROC:0.9931

Epoch: [69 | 1000] LR: 0.015872
Train | 8/8 | Loss:0.5357 | MainLoss:0.5050 | Alpha:0.4042 | SPLoss:0.0114 | CLSLoss:0.0644 | top1:75.9750 | AUROC:0.0000
Test | 65/8 | Loss:0.2233 | MainLoss:0.2233 | SPLoss:0.0115 | CLSLoss:0.0631 | top1:96.5047 | AUROC:0.9987
Test | 62/8 | Loss:0.3652 | MainLoss:0.3652 | SPLoss:0.0115 | CLSLoss:0.0631 | top1:85.4522 | AUROC:0.9933

Epoch: [70 | 1000] LR: 0.015868
Train | 8/8 | Loss:0.5242 | MainLoss:0.4945 | Alpha:0.4043 | SPLoss:0.0114 | CLSLoss:0.0620 | top1:78.2500 | AUROC:0.0000
Test | 65/8 | Loss:0.2260 | MainLoss:0.2260 | SPLoss:0.0113 | CLSLoss:0.0605 | top1:96.6916 | AUROC:0.9987
Test | 62/8 | Loss:0.3714 | MainL

Test | 65/8 | Loss:0.2219 | MainLoss:0.2219 | SPLoss:0.0145 | CLSLoss:0.0569 | top1:96.8474 | AUROC:0.9988
Test | 62/8 | Loss:0.3710 | MainLoss:0.3710 | SPLoss:0.0145 | CLSLoss:0.0569 | top1:84.8460 | AUROC:0.9935

Epoch: [91 | 1000] LR: 0.015755
Train | 8/8 | Loss:0.4651 | MainLoss:0.4355 | Alpha:0.4044 | SPLoss:0.0148 | CLSLoss:0.0583 | top1:79.5250 | AUROC:0.0000
Test | 65/8 | Loss:0.2246 | MainLoss:0.2246 | SPLoss:0.0154 | CLSLoss:0.0610 | top1:96.6885 | AUROC:0.9987
Test | 62/8 | Loss:0.3636 | MainLoss:0.3636 | SPLoss:0.0154 | CLSLoss:0.0610 | top1:85.3506 | AUROC:0.9935

Epoch: [92 | 1000] LR: 0.015749
Train | 8/8 | Loss:0.5767 | MainLoss:0.5469 | Alpha:0.4053 | SPLoss:0.0148 | CLSLoss:0.0585 | top1:74.0250 | AUROC:0.0000
Test | 65/8 | Loss:0.2267 | MainLoss:0.2267 | SPLoss:0.0141 | CLSLoss:0.0558 | top1:97.0031 | AUROC:0.9988
Test | 62/8 | Loss:0.3797 | MainLoss:0.3797 | SPLoss:0.0141 | CLSLoss:0.0558 | top1:84.2792 | AUROC:0.9936

Epoch: [93 | 1000] LR: 0.015742
Train | 8/8 | L

...

Test | 65/8 | Loss:0.2100 | MainLoss:0.2100 | SPLoss:0.0039 | CLSLoss:0.0440 | top1:96.0561 | AUROC:0.9990
Test | 62/8 | Loss:0.3325 | MainLoss:0.3325 | SPLoss:0.0039 | CLSLoss:0.0440 | top1:89.0531 | AUROC:0.9930

Epoch: [225 | 1000] LR: 0.014275
Train | 8/8 | Loss:0.4860 | MainLoss:0.4660 | Alpha:0.4083 | SPLoss:0.0041 | CLSLoss:0.0450 | top1:77.1750 | AUROC:0.0000
Test | 65/8 | Loss:0.2156 | MainLoss:0.2156 | SPLoss:0.0041 | CLSLoss:0.0440 | top1:95.9221 | AUROC:0.9990
Test | 62/8 | Loss:0.3310 | MainLoss:0.3310 | SPLoss:0.0041 | CLSLoss:0.0440 | top1:89.1809 | AUROC:0.9930

Epoch: [226 | 1000] LR: 0.014259
Train | 8/8 | Loss:0.5256 | MainLoss:0.5066 | Alpha:0.4087 | SPLoss:0.0042 | CLSLoss:0.0423 | top1:73.9500 | AUROC:0.0000
Test | 65/8 | Loss:0.2164 | MainLoss:0.2164 | SPLoss:0.0042 | CLSLoss:0.0420 | top1:96.3583 | AUROC:0.9990
Test | 62/8 | Loss:0.3469 | MainLoss:0.3469 | SPLoss:0.0042 | CLSLoss:0.0420 | top1:88.2667 | AUROC:0.9930

Epoch: [227 | 1000] LR: 0.014243
Train | 8/8 

Test | 62/8 | Loss:0.3402 | MainLoss:0.3402 | SPLoss:0.0058 | CLSLoss:0.0432 | top1:88.4142 | AUROC:0.9928

Epoch: [247 | 1000] LR: 0.013917
Train | 8/8 | Loss:0.4419 | MainLoss:0.4211 | Alpha:0.4091 | SPLoss:0.0059 | CLSLoss:0.0449 | top1:81.1000 | AUROC:0.0000
Test | 65/8 | Loss:0.2028 | MainLoss:0.2028 | SPLoss:0.0060 | CLSLoss:0.0475 | top1:95.7072 | AUROC:0.9991
Test | 62/8 | Loss:0.3140 | MainLoss:0.3140 | SPLoss:0.0060 | CLSLoss:0.0475 | top1:89.8231 | AUROC:0.9926

Epoch: [248 | 1000] LR: 0.013900
Train | 8/8 | Loss:0.4762 | MainLoss:0.4542 | Alpha:0.4084 | SPLoss:0.0061 | CLSLoss:0.0478 | top1:80.9750 | AUROC:0.0000
Test | 65/8 | Loss:0.1975 | MainLoss:0.1975 | SPLoss:0.0061 | CLSLoss:0.0474 | top1:96.4424 | AUROC:0.9991
Test | 62/8 | Loss:0.3329 | MainLoss:0.3329 | SPLoss:0.0061 | CLSLoss:0.0474 | top1:88.4830 | AUROC:0.9929

Epoch: [249 | 1000] LR: 0.013883
Train | 8/8 | Loss:0.4672 | MainLoss:0.4452 | Alpha:0.4080 | SPLoss:0.0062 | CLSLoss:0.0478 | top1:78.8000 | AUROC:0.00

Train | 8/8 | Loss:0.5426 | MainLoss:0.5233 | Alpha:0.4110 | SPLoss:0.0002 | CLSLoss:0.0470 | top1:76.2000 | AUROC:0.0000
Test | 65/8 | Loss:0.1972 | MainLoss:0.1972 | SPLoss:0.0002 | CLSLoss:0.0465 | top1:96.4704 | AUROC:0.9991
Test | 62/8 | Loss:0.3305 | MainLoss:0.3305 | SPLoss:0.0002 | CLSLoss:0.0465 | top1:88.6730 | AUROC:0.9927

Epoch: [270 | 1000] LR: 0.001351
Train | 8/8 | Loss:0.3888 | MainLoss:0.3695 | Alpha:0.4120 | SPLoss:0.0002 | CLSLoss:0.0468 | top1:85.3000 | AUROC:0.0000
Test | 65/8 | Loss:0.1954 | MainLoss:0.1954 | SPLoss:0.0002 | CLSLoss:0.0472 | top1:96.3770 | AUROC:0.9991
Test | 62/8 | Loss:0.3262 | MainLoss:0.3262 | SPLoss:0.0002 | CLSLoss:0.0472 | top1:88.8630 | AUROC:0.9928

Epoch: [271 | 1000] LR: 0.001349
Train | 8/8 | Loss:0.4656 | MainLoss:0.4460 | Alpha:0.4109 | SPLoss:0.0002 | CLSLoss:0.0473 | top1:83.0000 | AUROC:0.0000
Test | 65/8 | Loss:0.1954 | MainLoss:0.1954 | SPLoss:0.0002 | CLSLoss:0.0473 | top1:96.3209 | AUROC:0.9991
Test | 62/8 | Loss:0.3251 | Mai

Test | 65/8 | Loss:0.1990 | MainLoss:0.1990 | SPLoss:0.0004 | CLSLoss:0.0461 | top1:96.3022 | AUROC:0.9991
Test | 62/8 | Loss:0.3267 | MainLoss:0.3267 | SPLoss:0.0004 | CLSLoss:0.0461 | top1:89.0138 | AUROC:0.9928

Epoch: [292 | 1000] LR: 0.001310
Train | 8/8 | Loss:0.5229 | MainLoss:0.5040 | Alpha:0.4102 | SPLoss:0.0004 | CLSLoss:0.0458 | top1:73.1250 | AUROC:0.0000
Test | 65/8 | Loss:0.2003 | MainLoss:0.2003 | SPLoss:0.0004 | CLSLoss:0.0457 | top1:96.3115 | AUROC:0.9991
Test | 62/8 | Loss:0.3281 | MainLoss:0.3281 | SPLoss:0.0004 | CLSLoss:0.0457 | top1:88.9581 | AUROC:0.9928

Epoch: [293 | 1000] LR: 0.001308
Train | 8/8 | Loss:0.5055 | MainLoss:0.4866 | Alpha:0.4111 | SPLoss:0.0004 | CLSLoss:0.0456 | top1:77.5250 | AUROC:0.0000
Test | 65/8 | Loss:0.2017 | MainLoss:0.2017 | SPLoss:0.0004 | CLSLoss:0.0454 | top1:96.2928 | AUROC:0.9991
Test | 62/8 | Loss:0.3282 | MainLoss:0.3282 | SPLoss:0.0004 | CLSLoss:0.0454 | top1:89.0170 | AUROC:0.9927

Epoch: [294 | 1000] LR: 0.001306
Train | 8/8 

Test | 62/8 | Loss:0.3281 | MainLoss:0.3281 | SPLoss:0.0000 | CLSLoss:0.0459 | top1:88.8532 | AUROC:0.9926

Epoch: [314 | 1000] LR: 0.001266
